# Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012
from missingData.toolkits import toolkits

2025-03-16 15:06:59.005858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742148419.021109  242464 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742148419.026036  242464 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 15:06:59.043039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### StandardScaler

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-03-16 15:07:15 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-16 15:07:15 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-16 15:07:15 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-16 15:07:15 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-16 15:07:15 [INFO]: Loaded successfully!
2025-03-16 15:07:29 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 15:07:29 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 15:07:29 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

### MinMaxScaler

In [5]:
set_random_seed()
physionet2012_dataset_minmax = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)
print(physionet2012_dataset_minmax.keys())

2025-03-16 15:07:41 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-16 15:07:41 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-16 15:07:41 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-16 15:07:41 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-16 15:07:41 [INFO]: Loaded successfully!
2025-03-16 15:07:54 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 15:07:54 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-16 15:07:55 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [6]:
scaler_minmax = physionet2012_dataset_minmax["scaler"]

In [7]:
dataset_for_training_minmax, dataset_for_validating_minmax, dataset_for_testing_minmax, dataset_for_testing_ori_minmax = toolkits.separating_dataset(physionet2012_dataset_minmax)

# Creating indicating mask

### StandardScaler

In [8]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

In [9]:
dataset_for_testing_ori_standard_des = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_des = toolkits.dict_to_list(dataset_for_testing_standard)
dataset_for_testing_ori_standard_des = toolkits.pre_reshape(dataset_for_testing_ori_standard_des)
dataset_for_testing_standard_des = toolkits.pre_reshape(dataset_for_testing_standard_des)
dataset_for_testing_ori_standard_des = toolkits.desnormalization(dataset_for_testing_ori_standard_des, scaler_standard)
dataset_for_testing_standard_des = toolkits.desnormalization(dataset_for_testing_standard_des, scaler_standard)
indicating_mask_variable_standard_des, test_X_ori_variable_standard_des = toolkits.components_mae(dataset_for_testing_ori_standard_des, dataset_for_testing_standard_des)
indicating_mask_variable_standard_des = toolkits.reshape_variable(indicating_mask_variable_standard_des)
test_X_ori_variable_standard_des = toolkits.reshape_variable(test_X_ori_variable_standard_des)

### MinMaxScaler 

In [10]:
dataset_for_testing_ori_minmax_norm = toolkits.dict_to_list(dataset_for_testing_ori_minmax)
dataset_for_testing_minmax_norm = toolkits.dict_to_list(dataset_for_testing_minmax)
indicating_mask_variable_minmax_norm, test_X_ori_variable_minmax_norm = toolkits.components_mae(dataset_for_testing_ori_minmax_norm, dataset_for_testing_minmax_norm)
indicating_mask_variable_minmax_norm = toolkits.pre_reshape(indicating_mask_variable_minmax_norm)
test_X_ori_variable_minmax_norm = toolkits.pre_reshape(test_X_ori_variable_minmax_norm)
indicating_mask_variable_minmax_norm = toolkits.reshape_variable(indicating_mask_variable_minmax_norm)
test_X_ori_variable_minmax_norm = toolkits.reshape_variable(test_X_ori_variable_minmax_norm)

In [11]:
dataset_for_testing_ori_minmax_des = toolkits.dict_to_list(dataset_for_testing_ori_minmax)
dataset_for_testing_minmax_des = toolkits.dict_to_list(dataset_for_testing_minmax)
dataset_for_testing_ori_minmax_des = toolkits.pre_reshape(dataset_for_testing_ori_minmax_des)
dataset_for_testing_minmax_des = toolkits.pre_reshape(dataset_for_testing_minmax_des)
dataset_for_testing_ori_minmax_des = toolkits.desnormalization(dataset_for_testing_ori_minmax_des, scaler_minmax)
dataset_for_testing_minmax_des = toolkits.desnormalization(dataset_for_testing_minmax_des, scaler_minmax)
indicating_mask_variable_minmax_des, test_X_ori_variable_minmax_des = toolkits.components_mae(dataset_for_testing_ori_minmax_des, dataset_for_testing_minmax_des)
indicating_mask_variable_minmax_des = toolkits.reshape_variable(indicating_mask_variable_minmax_des)
test_X_ori_variable_minmax_des = toolkits.reshape_variable(test_X_ori_variable_minmax_des)

# Inicialize the models

### SAITS

<h4> Inicialize new model (Standard)</h4>

In [12]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 15:11:15 [INFO]: No given device, using default device: cpu
2025-03-16 15:11:15 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250316_T151115
2025-03-16 15:11:15 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250316_T151115/tensorboard
2025-03-16 15:11:15 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (Standard)</h4>

In [7]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-16 14:52:27 [INFO]: No given device, using default device: cpu
2025-03-16 14:52:27 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 14:52:27 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize new model (MinMax)</h4>

In [13]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-16 15:11:25 [INFO]: No given device, using default device: cpu
2025-03-16 15:11:25 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250316_T151125
2025-03-16 15:11:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250316_T151125/tensorboard
2025-03-16 15:11:25 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

### BRITS

<h4> Inicialize new model (Standard)</h4>

In [16]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 22:14:19 [INFO]: No given device, using default device: cpu
2025-03-14 22:14:19 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250314_T221419
2025-03-14 22:14:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250314_T221419/tensorboard
2025-03-14 22:14:19 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-02-25 21:04:33 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:33 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:33 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize new model (MinMax)</h4>

In [17]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 22:14:24 [INFO]: No given device, using default device: cpu
2025-03-14 22:14:24 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250314_T221424
2025-03-14 22:14:24 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250314_T221424/tensorboard
2025-03-14 22:14:24 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

### USGAN

<h4> Inicialize new model (Standard)</h4>

In [20]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 22:30:35 [INFO]: No given device, using default device: cpu
2025-03-14 22:30:35 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250314_T223035
2025-03-14 22:30:35 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250314_T223035/tensorboard
2025-03-14 22:30:36 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-02-25 21:04:37 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:37 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:37 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize new model (MinMax)</h4>

In [21]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 22:30:41 [INFO]: No given device, using default device: cpu
2025-03-14 22:30:41 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250314_T223041
2025-03-14 22:30:41 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250314_T223041/tensorboard
2025-03-14 22:30:41 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

### GPVAE

<h4> Inicialize new model (Standard)</h4>

In [24]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 23:04:32 [INFO]: No given device, using default device: cpu
2025-03-14 23:04:32 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250314_T230432
2025-03-14 23:04:32 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250314_T230432/tensorboard
2025-03-14 23:04:32 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-02-25 21:04:42 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:42 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:42 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize new model (MinMax)</h4>

In [25]:
gp_vae_minmax = GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 23:04:44 [INFO]: No given device, using default device: cpu
2025-03-14 23:04:44 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250314_T230444
2025-03-14 23:04:44 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250314_T230444/tensorboard
2025-03-14 23:04:44 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
gp_vae_minmax= GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

### MRNN

<h4> Inicialize new model (Standard)</h4>

In [28]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 23:18:57 [INFO]: No given device, using default device: cpu
2025-03-14 23:18:57 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250314_T231857
2025-03-14 23:18:57 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250314_T231857/tensorboard
2025-03-14 23:18:57 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-02-25 21:04:45 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:45 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:45 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize new model (MinMax)</h4>

In [29]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

2025-03-14 23:19:01 [INFO]: No given device, using default device: cpu
2025-03-14 23:19:01 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250314_T231901
2025-03-14 23:19:01 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250314_T231901/tensorboard
2025-03-14 23:19:01 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="all",
)

# Train/Load the models

### SAITS

<h4>Train - Standard Scaler </h4>

In [14]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-16 15:11:49 [INFO]: Epoch 001 - training loss (MSE): 0.8186, validation MSE: 6.7439
2025-03-16 15:11:59 [INFO]: Epoch 002 - training loss (MSE): 0.5314, validation MSE: 6.6840
2025-03-16 15:12:08 [INFO]: Epoch 003 - training loss (MSE): 0.5029, validation MSE: 6.6729
2025-03-16 15:12:18 [INFO]: Epoch 004 - training loss (MSE): 0.4546, validation MSE: 6.6573
2025-03-16 15:12:28 [INFO]: Epoch 005 - training loss (MSE): 0.4356, validation MSE: 6.6584
2025-03-16 15:12:37 [INFO]: Epoch 006 - training loss (MSE): 0.4727, validation MSE: 6.6585
2025-03-16 15:12:47 [INFO]: Epoch 007 - training loss (MSE): 0.4030, validation MSE: 6.6770
2025-03-16 15:12:47 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-16 15:12:47 [INFO]: Finished training. The best model is from epoch#4.
2025-03-16 15:12:47 [INFO]: Saved the model to tutorial_results/imputation/saits/20250316_T151115/SAITS.pypots


<h4>Train - MinMax Scaler</h4>

In [15]:
saits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-14 22:11:19 [INFO]: Epoch 001 - training loss (MSE): 0.0360, validation MSE: 0.0082
2025-03-14 22:11:19 [INFO]: Saved the model to tutorial_results/imputation/saits/20250314_T220832/SAITS_epoch1_loss0.0082.pypots
2025-03-14 22:11:29 [INFO]: Epoch 002 - training loss (MSE): 0.0163, validation MSE: 0.0064
2025-03-14 22:11:29 [INFO]: Saved the model to tutorial_results/imputation/saits/20250314_T220832/SAITS_epoch2_loss0.0064.pypots
2025-03-14 22:11:38 [INFO]: Epoch 003 - training loss (MSE): 0.0124, validation MSE: 0.0056
2025-03-14 22:11:38 [INFO]: Saved the model to tutorial_results/imputation/saits/20250314_T220832/SAITS_epoch3_loss0.0056.pypots
2025-03-14 22:11:47 [INFO]: Epoch 004 - training loss (MSE): 0.0100, validation MSE: 0.0045
2025-03-14 22:11:47 [INFO]: Saved the model to tutorial_results/imputation/saits/20250314_T220832/SAITS_epoch4_loss0.0045.pypots
2025-03-14 22:11:57 [INFO]: Epoch 005 - training loss (MSE): 0.0083, validation MSE: 0.0039
2025-03-14 22:11:57 [INF

<h4> Load - Standard Scaler </h4>

In [9]:
saits_standard.load("tutorial_results/imputation/saits/20250316_T145645/SAITS.pypots")

2025-03-16 14:59:36 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250316_T145645/SAITS.pypots


<h4> Load - Min/Max Scaler </h4>

In [ ]:
saits_minmax.load("tutorial_results/imputation/saits/minmax/SAITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-2

### BRITS

<h4> Train - Standard Scaler </h4>

In [18]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-14 22:15:43 [INFO]: Epoch 001 - training loss (MSE): 0.9450, validation MSE: 6.7843
2025-03-14 22:15:43 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221419/BRITS_epoch1_loss6.7843.pypots
2025-03-14 22:16:20 [INFO]: Epoch 002 - training loss (MSE): 0.7358, validation MSE: 6.7442
2025-03-14 22:16:20 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221419/BRITS_epoch2_loss6.7442.pypots
2025-03-14 22:16:57 [INFO]: Epoch 003 - training loss (MSE): 0.6841, validation MSE: 6.7316
2025-03-14 22:16:57 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221419/BRITS_epoch3_loss6.7316.pypots
2025-03-14 22:17:35 [INFO]: Epoch 004 - training loss (MSE): 0.6600, validation MSE: 6.7264
2025-03-14 22:17:35 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221419/BRITS_epoch4_loss6.7264.pypots
2025-03-14 22:18:12 [INFO]: Epoch 005 - training loss (MSE): 0.6447, validation MSE: 6.7258
2025-03-14 22:18:12 [INF

<h4> Train - MinMax Scaler</h4>

In [19]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-14 22:22:08 [INFO]: Epoch 001 - training loss (MSE): 0.1666, validation MSE: 0.0076
2025-03-14 22:22:08 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221424/BRITS_epoch1_loss0.0076.pypots
2025-03-14 22:22:45 [INFO]: Epoch 002 - training loss (MSE): 0.0979, validation MSE: 0.0053
2025-03-14 22:22:45 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221424/BRITS_epoch2_loss0.0053.pypots
2025-03-14 22:23:22 [INFO]: Epoch 003 - training loss (MSE): 0.0829, validation MSE: 0.0045
2025-03-14 22:23:22 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221424/BRITS_epoch3_loss0.0045.pypots
2025-03-14 22:23:59 [INFO]: Epoch 004 - training loss (MSE): 0.0759, validation MSE: 0.0039
2025-03-14 22:23:59 [INFO]: Saved the model to tutorial_results/imputation/brits/20250314_T221424/BRITS_epoch4_loss0.0039.pypots
2025-03-14 22:24:36 [INFO]: Epoch 005 - training loss (MSE): 0.0707, validation MSE: 0.0035
2025-03-14 22:24:36 [INF

<h4>Load - Standard Scaler</h4>

In [ ]:
brits_standard.load("tutorial_results/imputation/brits/standard/BRITS.pypots")

2025-02-25 21:05:09 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard_scaler/BRITS.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
brits_minmax.load("tutorial_results/imputation/brits/minmax/BRITS.pypots")

NameError: name 'brits' is not defined

### USGAN

<h4>Train - Standard Scaler </h4>

In [22]:
us_gan_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-14 22:32:37 [INFO]: Epoch 001 - generator training loss: 0.4208, discriminator training loss: 0.1856, validation loss: 6.7608
2025-03-14 22:32:37 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223035/USGAN_epoch1_loss6.7608.pypots
2025-03-14 22:34:03 [INFO]: Epoch 002 - generator training loss: 0.3496, discriminator training loss: 0.0542, validation loss: 6.7149
2025-03-14 22:34:03 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223035/USGAN_epoch2_loss6.7149.pypots
2025-03-14 22:35:29 [INFO]: Epoch 003 - generator training loss: 0.3237, discriminator training loss: 0.0362, validation loss: 6.7062
2025-03-14 22:35:29 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223035/USGAN_epoch3_loss6.7062.pypots
2025-03-14 22:36:56 [INFO]: Epoch 004 - generator training loss: 0.3136, discriminator training loss: 0.0306, validation loss: 6.6885
2025-03-14 22:36:56 [INFO]: Saved the model to tutorial_results/imputation

<h4> Train - MinMax Scaler</h4>

In [23]:
us_gan_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-14 22:49:15 [INFO]: Epoch 001 - generator training loss: -0.0580, discriminator training loss: 0.1818, validation loss: 0.0209
2025-03-14 22:49:15 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223041/USGAN_epoch1_loss0.0209.pypots
2025-03-14 22:50:42 [INFO]: Epoch 002 - generator training loss: -0.0013, discriminator training loss: 0.0539, validation loss: 0.0140
2025-03-14 22:50:42 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223041/USGAN_epoch2_loss0.0140.pypots
2025-03-14 22:52:08 [INFO]: Epoch 003 - generator training loss: 0.0013, discriminator training loss: 0.0377, validation loss: 0.0125
2025-03-14 22:52:08 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250314_T223041/USGAN_epoch3_loss0.0125.pypots
2025-03-14 22:53:36 [INFO]: Epoch 004 - generator training loss: -0.0007, discriminator training loss: 0.0319, validation loss: 0.0127
2025-03-14 22:53:36 [INFO]: Saved the model to tutorial_results/imputat

<h4>Load - Standard Scaler</h4>

In [ ]:
us_gan_standard.load("tutorial_results/imputation/us_gan/standard/USGAN.pypots")

2025-02-25 21:05:34 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/standard_scaler/USGAN.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
us_gan_minmax.load("tutorial_results/imputation/us_gan/minmax/USGAN.pypots")

### GPVAE

<h4>Train - Standard Scaler</h4>

In [26]:
gp_vae_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-14 23:05:22 [INFO]: Epoch 001 - training loss (MSE): 26005.7651, validation MSE: 7.0331
2025-03-14 23:05:22 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230432/GPVAE_epoch1_loss7.0331.pypots
2025-03-14 23:05:40 [INFO]: Epoch 002 - training loss (MSE): 22872.9500, validation MSE: 7.0307
2025-03-14 23:05:40 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230432/GPVAE_epoch2_loss7.0307.pypots
2025-03-14 23:05:59 [INFO]: Epoch 003 - training loss (MSE): 22841.9535, validation MSE: 6.9962
2025-03-14 23:05:59 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230432/GPVAE_epoch3_loss6.9962.pypots
2025-03-14 23:06:18 [INFO]: Epoch 004 - training loss (MSE): 22833.0582, validation MSE: 6.9798
2025-03-14 23:06:18 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230432/GPVAE_epoch4_loss6.9798.pypots
2025-03-14 23:06:36 [INFO]: Epoch 005 - training loss (MSE): 22824.8062, validation MSE: 6.9814


<h4>Train - MinMax Scaler</h4>

In [27]:
gp_vae_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-14 23:12:00 [INFO]: Epoch 001 - training loss (MSE): 24751.6873, validation MSE: 0.0238
2025-03-14 23:12:00 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230444/GPVAE_epoch1_loss0.0238.pypots
2025-03-14 23:12:22 [INFO]: Epoch 002 - training loss (MSE): 22721.8431, validation MSE: 0.0111
2025-03-14 23:12:22 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230444/GPVAE_epoch2_loss0.0111.pypots
2025-03-14 23:12:46 [INFO]: Epoch 003 - training loss (MSE): 22712.9646, validation MSE: 0.0110
2025-03-14 23:12:46 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230444/GPVAE_epoch3_loss0.0110.pypots
2025-03-14 23:13:10 [INFO]: Epoch 004 - training loss (MSE): 22710.9633, validation MSE: 0.0108
2025-03-14 23:13:10 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250314_T230444/GPVAE_epoch4_loss0.0108.pypots
2025-03-14 23:13:34 [INFO]: Epoch 005 - training loss (MSE): 22710.2003, validation MSE: 0.0109


<h4>Load - Standard Scaler</h4>

In [ ]:
gp_vae_standard.load("tutorial_results/imputation/gp_vae/standard/GPVAE.pypots")

2025-02-25 21:05:46 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/standard_scaler/GPVAE.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
gp_vae_minmax.load("mae/tutorial_results/imputation/gp_vae/minmax/GPVAE.pypots")

### MRNN

<h4>Train - Standard Scaler</h4>

In [30]:
mrnn_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-14 23:25:31 [INFO]: Epoch 001 - training loss (MSE): 0.7082, validation MSE: 7.3078
2025-03-14 23:25:31 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231857/MRNN_epoch1_loss7.3078.pypots
2025-03-14 23:31:32 [INFO]: Epoch 002 - training loss (MSE): 0.5308, validation MSE: 7.2712
2025-03-14 23:31:32 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231857/MRNN_epoch2_loss7.2712.pypots
2025-03-14 23:37:29 [INFO]: Epoch 003 - training loss (MSE): 0.4895, validation MSE: 7.2554
2025-03-14 23:37:29 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231857/MRNN_epoch3_loss7.2554.pypots
2025-03-14 23:43:30 [INFO]: Epoch 004 - training loss (MSE): 0.4704, validation MSE: 7.2477
2025-03-14 23:43:30 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231857/MRNN_epoch4_loss7.2477.pypots
2025-03-14 23:49:25 [INFO]: Epoch 005 - training loss (MSE): 0.4565, validation MSE: 7.2435
2025-03-14 23:49:25 [INFO]: Save

<h4>Train - MinMax Scaler</h4>

In [31]:
mrnn_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-15 00:15:40 [INFO]: Epoch 001 - training loss (MSE): 0.2016, validation MSE: 0.0279
2025-03-15 00:15:40 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231901/MRNN_epoch1_loss0.0279.pypots
2025-03-15 00:21:41 [INFO]: Epoch 002 - training loss (MSE): 0.0690, validation MSE: 0.0340
2025-03-15 00:21:41 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231901/MRNN_epoch2_loss0.0340.pypots
2025-03-15 00:27:44 [INFO]: Epoch 003 - training loss (MSE): 0.0536, validation MSE: 0.0429
2025-03-15 00:27:44 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231901/MRNN_epoch3_loss0.0429.pypots
2025-03-15 00:33:46 [INFO]: Epoch 004 - training loss (MSE): 0.0449, validation MSE: 0.0490
2025-03-15 00:33:46 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250314_T231901/MRNN_epoch4_loss0.0490.pypots
2025-03-15 00:33:46 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-03-15 00:33:46 [INFO]

<h4>Load - Standard Scaler</h4>

In [ ]:
mrnn_standard.load("tutorial_results/imputation/mrnn/standard/MRNN.pypots")

2025-02-25 21:05:52 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/standard_scaler/MRNN.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
mrnn_minmax.load("tutorial_results/imputation/mrnn/minmax/MRNN.pypots")

# Testing stage

### SAITS

#### Standard Scaler

In [10]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

#### MinMax Scaler

In [17]:
saits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, saits_minmax)
saits_imputation_minmax = toolkits.pre_reshape(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.desnormalization(saits_imputation_minmax, scaler_minmax)
saits_imputation_minmax = toolkits.reshape_variable(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.reshape_variable(saits_imputation_minmax_ori)

### BRITS

#### Standard Scaler

In [32]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits_standard)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

#### MinMax Scaler

In [33]:
brits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, brits_minmax)
brits_imputation_minmax = toolkits.pre_reshape(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax) 
brits_imputation_minmax = toolkits.reshape_variable(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.reshape_variable(brits_imputation_minmax_ori)  

### USGAN

#### Standard Scaler

In [34]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan_standard)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

#### MinMax Scaler

In [36]:
usgan_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, us_gan_minmax)
usgan_imputation_minmax = toolkits.pre_reshape(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.desnormalization(usgan_imputation_minmax, scaler_minmax)
usgan_imputation_minmax = toolkits.reshape_variable(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.reshape_variable(usgan_imputation_minmax_ori)

### GPVAE

#### Standard Scaler

In [37]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae_standard)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori) 

#### MinMax Scaler

In [38]:
gpvae_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, gp_vae_minmax)
gpvae_imputation_minmax = toolkits.pre_reshape(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.desnormalization(gpvae_imputation_minmax, scaler_minmax)
gpvae_imputation_minmax = toolkits.reshape_variable(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.reshape_variable(gpvae_imputation_minmax_ori) 

### MRNN

#### Standard Scaler

In [39]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn_standard)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori) 

#### MinMax Scaler

In [40]:
mrnn_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, mrnn_minmax)
mrnn_imputation_minmax = toolkits.pre_reshape(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.desnormalization(mrnn_imputation_minmax, scaler_minmax)
mrnn_imputation_minmax = toolkits.reshape_variable(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.reshape_variable(mrnn_imputation_minmax_ori) 

# Calculate mean absolute error

### SAITS

#### Standard scaler (C/Normalização)

In [11]:
testing_mae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

#### Standard scaler (S/Normalização)

In [12]:
testing_mae_saits_variables_standard_ori = toolkits.calculate_mae(saits_imputation_standard_ori, test_X_ori_variable_standard_des, indicating_mask_variable_standard_des)

#### MinMax scaler (C/Normalização)

In [20]:
testing_mae_saits_variables_minmax = toolkits.calculate_mae(saits_imputation_minmax, test_X_ori_variable_minmax_norm, indicating_mask_variable_minmax_norm)

#### MinMax scaler (S/Normalização)

In [21]:
testing_mae_saits_variables_minmax_ori = toolkits.calculate_mae(saits_imputation_minmax_ori, test_X_ori_variable_minmax_des, indicating_mask_variable_minmax_des)

### BRITS

#### Standard Scaler (C/Normalização)

In [45]:
testing_mae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [46]:
testing_mae_brits_variables_standard_ori = toolkits.calculate_mae(brits_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [47]:
testing_mae_brits_variables_minmax = toolkits.calculate_mae(brits_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [48]:
testing_mae_brits_variables_minmax_ori = toolkits.calculate_mae(brits_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### USGAN

#### Standard Scaler (C/Normalização)

In [49]:
testing_mae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [50]:
testing_mae_usgan_variables_standard_ori = toolkits.calculate_mae(usgan_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [51]:
testing_mae_usgan_variables_minmax = toolkits.calculate_mae(usgan_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [52]:
testing_mae_usgan_variables_minmax_ori = toolkits.calculate_mae(usgan_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### GPVAE

#### Standard Scaler (C/Normalização)

In [53]:
testing_mae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [54]:
testing_mae_gpvae_variables_standard_ori = toolkits.calculate_mae(gpvae_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [55]:
testing_mae_gpvae_variables_minmax = toolkits.calculate_mae(gpvae_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [56]:
testing_mae_gpvae_variables_minmax_ori = toolkits.calculate_mae(gpvae_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### MRNN

#### Standard Scaler (C/Normalização)

In [57]:
testing_mae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [58]:
testing_mae_mrnn_variables_standard_ori = toolkits.calculate_mae(mrnn_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [59]:
testing_mae_mrnn_variables_minmax = toolkits.calculate_mae(mrnn_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [60]:
testing_mae_mrnn_variables_minmax_ori = toolkits.calculate_mae(mrnn_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

# Results mean absolute error

In [13]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [14]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [15]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35842167090154575
ALT : 0.22873105156973475
AST : 0.2360348751195685
Albumin : 0.5669722943273814
BUN : 0.21043372275375152
Bilirubin : 0.2799269029920604
Cholesterol : 0.8929161650373507
Creatinine : 0.1904952408486541
DiasABP : 0.20309505135870332
FiO2 : 0.46299985373247476
GCS : 0.25740588128546604
Glucose : 0.58622658054476
HCO3 : 0.4138240084389746
HCT : 0.5282681467592029
HR : 0.26094783579023706
K : 0.6036355663482271
Lactate : 0.3863171737590092
MAP : 0.17923546222764594
MechVent : 0.010012425656927714
Mg : 0.5337098468198467
NIDiasABP : 0.2158271249873218
NIMAP : 0.142324202564339
NISysABP : 0.24474839151507188
Na : 0.41931335367054434
PaCO2 : 0.5161239232628241
PaO2 : 0.5040665952956557
Platelets : 0.36926333356189006
RespRate : 0.461044411387632
SaO2 : 0.353425374402371
SysABP : 0.24250818731396934
Temp : 0.32115819242427524
TroponinI : 0.6336863382590735
TroponinT : 0.35103726598576673
Urine : 0.3666884967716744
WBC : 0

In [25]:
df_saits_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [26]:
df_saits_mae_standard

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.355492,0.398791,0.388989,0.000000,0.331751,0.740879,0.222358,0.195952,0.723086,0.505640,0.296971,0.216395,0.284060,0.236059,0.184009,0.347203,0.266653
1,ALT,0.346164,0.267721,0.253539,0.000000,0.280160,0.329866,0.296968,0.351267,0.311489,0.272558,0.325780,0.238700,0.251692,0.305938,0.274798,0.318639,0.823624
2,AST,0.283204,0.257204,0.204749,0.000000,0.206225,0.243144,0.340113,0.273045,0.305834,0.306141,0.244520,0.699315,0.218977,0.318423,0.207619,0.237748,0.209382
3,Albumin,0.586538,0.609286,0.621456,0.000000,0.536556,0.658661,0.490084,0.717311,0.708582,0.754085,0.776171,1.172093,0.840292,0.814925,0.467807,0.670209,0.729714
4,BUN,0.235495,0.253363,0.228878,0.000000,0.242391,0.244288,0.213308,0.222898,0.294829,0.209661,0.257893,0.200723,0.219590,0.235670,0.210442,0.309354,0.251667
5,Bilirubin,0.292134,0.301597,0.249420,0.000000,0.231827,0.336409,0.200403,0.318250,0.280385,0.267887,0.312005,0.954099,0.392959,0.242916,0.335206,0.310083,0.216768
6,Cholesterol,0.861914,1.451439,0.918617,0.000000,0.736495,1.183795,1.002457,0.000000,0.906296,0.725019,0.618523,0.085870,0.704775,1.232571,0.533449,0.347422,0.000000
7,Creatinine,0.227236,0.218419,0.210438,0.000000,0.200026,0.211177,0.193449,0.183120,0.265242,0.217446,0.233475,0.177189,0.227362,0.206401,0.179911,0.192612,0.192658
8,DiasABP,0.256964,0.260941,0.240924,0.182851,0.271154,0.231552,0.299906,0.221175,0.250333,0.241564,0.250752,0.246645,0.246105,0.255920,0.251888,0.235695,0.250253
9,FiO2,0.484346,0.452276,0.517671,0.000000,0.491061,0.501327,0.482516,0.467761,0.507122,0.438264,0.460432,0.476691,0.439314,0.414027,0.507958,0.501128,0.573521


In [44]:
latex_code = df_saits_mae_standard.to_latex(index=False)

with open("tabela.tex", "w") as f:
    f.write(latex_code)

In [45]:
print(latex_code)

\begin{tabular}{lrrrrrrrrrrrrrrrrr}
\toprule
0 & General & Female & Male & Undefined Gender & +65 & -65 & ICUType 1 & ICUType 2 & ICUType 3 & ICUType 4 & Undefined classification & Low Weight & Normal Weight & Overweight & Obesity 1 & Obesity 2 & Obesity 3 \\
\midrule
ALP & 0.355492 & 0.398791 & 0.388989 & 0.000000 & 0.331751 & 0.740879 & 0.222358 & 0.195952 & 0.723086 & 0.505640 & 0.296971 & 0.216395 & 0.284060 & 0.236059 & 0.184009 & 0.347203 & 0.266653 \\
ALT & 0.346164 & 0.267721 & 0.253539 & 0.000000 & 0.280160 & 0.329866 & 0.296968 & 0.351267 & 0.311489 & 0.272558 & 0.325780 & 0.238700 & 0.251692 & 0.305938 & 0.274798 & 0.318639 & 0.823624 \\
AST & 0.283204 & 0.257204 & 0.204749 & 0.000000 & 0.206225 & 0.243144 & 0.340113 & 0.273045 & 0.305834 & 0.306141 & 0.244520 & 0.699315 & 0.218977 & 0.318423 & 0.207619 & 0.237748 & 0.209382 \\
Albumin & 0.586538 & 0.609286 & 0.621456 & 0.000000 & 0.536556 & 0.658661 & 0.490084 & 0.717311 & 0.708582 & 0.754085 & 0.776171 & 1.172093 & 0.84029

<h5>Minimum MAE value in each subgroup</h5>

In [27]:
toolkits.min_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
MechVent
0.016131509283529688
--------------------
Female
MechVent
0.014552340041130598
--------------------
Male
MechVent
0.014512000309648741
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.012942020909290175
--------------------
-65
MechVent
0.01827778097688463
--------------------
ICUType 1
MechVent
0.012979781499681835
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.015398824789147038
--------------------
ICUType 4
MechVent
0.01676655673093299
--------------------
Undefined classification
MechVent
0.015602762801210616
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.013784697335756175
--------------------
Overweight
MechVent
0.014702130836156856
--------------------
Obesity 1
MechVent
0.014115385650734221
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [28]:
toolkits.max_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
Cholesterol
0.8619142944054452
--------------------
Female
Cholesterol
1.451438736858023
--------------------
Male
Cholesterol
0.9186172160150028
--------------------
Undefined Gender
K
0.6481980955579346
--------------------
+65
TroponinI
0.912479961059112
--------------------
-65
Cholesterol
1.1837952493687076
--------------------
ICUType 1
TroponinI
1.4147481358654532
--------------------
ICUType 2
Albumin
0.7173112394561357
--------------------
ICUType 3
SaO2
1.5843908865861114
--------------------
ICUType 4
TroponinI
0.8606517177368178
--------------------
Undefined classification
Albumin
0.7761709655447225
--------------------
Low Weight
Albumin
1.1720931746200332
--------------------
Normal Weight
Albumin
0.8402915751610249
--------------------
Overweight
Cholesterol
1.2325708781179912
--------------------
Obesity 1
Ph
1.006255796754588
--------------------
Obesity 2
TroponinT
1.4516069413777122
--------------------
Obesity 3
TroponinT
1.03692539683769
------------------

<h4>Standard Scaler (S/Normalização)</h4>

In [29]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard_ori, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 46.05178056160585
ALT : 406.2112769981204
AST : 496.992055665294
Albumin : 0.3851788459064264
BUN : 5.380332453297799
Bilirubin : 1.6736765518848495
Cholesterol : 38.400731277463905
Creatinine : 0.3503998682295476
DiasABP : 3.286241163141612
FiO2 : 0.09104833207391178
GCS : 1.1115600990723302
Glucose : 35.67556060432917
HCO3 : 1.9973297300677588
HCT : 2.7491760875685074
HR : 4.954845594432205
K : 0.4188106400472619
Lactate : 0.9578556416874965
MAP : 4.378979286697319
MechVent : 0.01613150967851928
Mg : 0.2635250708242739
NIDiasABP : 3.5702379063267773
NIMAP : 2.6413458948607875
NISysABP : 6.786345809909647
Na : 2.6723842429156246
PaCO2 : 4.967970567426776
PaO2 : 43.05042321078405
Platelets : 40.793265397332874
RespRate : 2.670931693010336
SaO2 : 1.3594024396249635
SysABP : 6.740216718958554
Temp : 0.466368066859829
TroponinI : 6.362012470364254
TroponinT : 1.6937565297072215
Urine : 71.26128697044311
WBC : 6.241771727095261
Weight : 

In [30]:
df_saits_mae_standard_ori = toolkits.create_table(testing_mae_saits_variables_standard_ori, subgroups, variables)

In [31]:
df_saits_mae_standard_ori

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,46.051781,51.660965,50.391101,0.000000,42.976266,95.976280,28.805102,25.384391,93.671351,65.502523,38.470790,28.032644,36.798283,30.579934,23.837229,44.978069,34.543202
1,ALT,406.211277,314.161357,297.518824,0.000000,328.757487,387.086329,348.481409,412.199358,365.521484,319.837259,382.291076,280.105937,295.352176,359.007188,322.465492,373.911811,966.493927
2,AST,496.992056,451.364648,359.312336,0.000000,361.902289,426.691253,596.861957,479.165045,536.706173,537.244698,429.105660,1227.220856,384.279929,558.798516,364.348525,417.221308,367.442741
3,Albumin,0.385179,0.400118,0.408109,0.000000,0.352356,0.432542,0.321838,0.471058,0.465325,0.495207,0.509711,0.769713,0.551819,0.535161,0.307209,0.440125,0.479202
4,BUN,5.380332,5.788564,5.229164,0.000000,5.537895,5.581244,4.873432,5.092528,6.735940,4.790106,5.892070,4.585910,5.016969,5.384336,4.807965,7.067806,5.749823
5,Bilirubin,1.673677,1.727891,1.428961,0.000000,1.328168,1.927334,1.148139,1.823300,1.606364,1.534762,1.787523,5.466168,2.251319,1.391699,1.920443,1.776509,1.241894
6,Cholesterol,38.400731,64.665720,40.927007,0.000000,32.812929,52.741440,44.662291,0.000000,40.378078,32.301643,27.556951,3.825745,31.399719,54.914536,23.766652,15.478622,0.000000
7,Creatinine,0.350400,0.336804,0.324496,0.000000,0.308442,0.325636,0.298301,0.282373,0.409005,0.335304,0.360020,0.273227,0.350594,0.318273,0.277424,0.297009,0.297080
8,DiasABP,3.286241,3.337108,3.081119,2.338437,3.467723,2.961258,3.835419,2.828557,3.201445,3.089303,3.206805,3.154283,3.147369,3.272900,3.221336,3.014249,3.200419
9,FiO2,0.091048,0.085020,0.097313,0.000000,0.092311,0.094241,0.090704,0.087931,0.095330,0.082386,0.086553,0.089609,0.082583,0.077830,0.095487,0.094203,0.107812


<h5>Minimum MAE value in each subgroup</h5>

In [32]:
toolkits.min_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

General
MechVent
0.01613150967851928
--------------------
Female
MechVent
0.01455234083128562
--------------------
Male
MechVent
0.01451199920706472
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.012942019615087409
--------------------
-65
MechVent
0.01827777949621049
--------------------
ICUType 1
MechVent
0.01297977845445802
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.015398822799574146
--------------------
ICUType 4
MechVent
0.016766555871361
--------------------
Undefined classification
MechVent
0.015602763073054438
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.01378469708507042
--------------------
Overweight
MechVent
0.01470213082798736
--------------------
Obesity 1
MechVent
0.014115385917010151
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [33]:
toolkits.max_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

General
AST
496.992055665294
--------------------
Female
AST
451.3646475829517
--------------------
Male
AST
359.3123356241838
--------------------
Undefined Gender
Urine
65.78923797600842
--------------------
+65
AST
361.9022886474364
--------------------
-65
AST
426.6912528872452
--------------------
ICUType 1
AST
596.8619574737311
--------------------
ICUType 2
AST
479.1650450333097
--------------------
ICUType 3
AST
536.7061729221494
--------------------
ICUType 4
AST
537.2446980476282
--------------------
Undefined classification
AST
429.10566036324303
--------------------
Low Weight
AST
1227.220855712584
--------------------
Normal Weight
AST
384.2799291610597
--------------------
Overweight
AST
558.7985157739283
--------------------
Obesity 1
AST
364.3485252380189
--------------------
Obesity 2
AST
417.2213083901604
--------------------
Obesity 3
ALT
966.4939270016309
--------------------


<h4>MinMax Scaler (C/Normalização)</h4>

In [34]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.040219907134693526
ALT : 0.017229766448887466
AST : 0.015468583460460567
Albumin : 0.45373933730150834
BUN : 0.11917518228086253
Bilirubin : 0.04461143951106635
Cholesterol : 0.38712575286624884
Creatinine : 0.06330636563215755
DiasABP : 0.2222883897032941
FiO2 : 0.4212544503171473
GCS : 0.7013794357043793
Glucose : 0.08704999971751116
HCO3 : 0.38295172317766624
HCT : 0.45079366319242514
HR : 0.43347807748668743
K : 0.12318451615964963
Lactate : 0.08926922807197073
MAP : 0.2694956399200512
MechVent : 0.0
Mg : 0.06937552593284982
NIDiasABP : 0.34968573475266584
NIMAP : 0.34092245932783305
NISysABP : 0.45429145035202567
Na : 0.4986517947522833
PaCO2 : 0.4095278180110242
PaO2 : 0.29658351907452635
Platelets : 0.08119226622028226
RespRate : 0.19828358249685923
SaO2 : 0.9685085946654024
SysABP : 0.42148474257749563
Temp : 0.9335184893617962
TroponinI : 0.16517311730421255
TroponinT : 0.02879763797807984
Urine : 0.010554451556121645
WBC 

In [35]:
df_saits_mae_minmax = toolkits.create_table(testing_mae_saits_variables_minmax, subgroups, variables)

In [36]:
df_saits_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.040220,0.041253,0.042669,0.000000,0.045370,0.058368,0.029260,0.018007,0.063298,0.040524,0.036990,0.128495,0.039369,0.032560,0.033538,0.019912,0.031346
1,ALT,0.017230,0.006941,0.016567,0.000000,0.012877,0.023744,0.018795,0.029335,0.023352,0.016529,0.017182,0.012010,0.009226,0.037875,0.014711,0.045587,0.099730
2,AST,0.015469,0.017081,0.006773,0.000000,0.006469,0.011712,0.018681,0.008078,0.013382,0.017210,0.007568,0.024316,0.004764,0.018236,0.010571,0.002253,0.001133
3,Albumin,0.453739,0.423997,0.450085,0.000000,0.428799,0.450342,0.544186,0.392027,0.419730,0.411677,0.404592,0.488372,0.380661,0.470012,0.461240,0.406977,0.390698
4,BUN,0.119175,0.115023,0.134813,0.000000,0.147150,0.102920,0.136926,0.092971,0.158645,0.091490,0.125755,0.143510,0.128078,0.122560,0.140897,0.149504,0.141484
5,Bilirubin,0.044611,0.034648,0.036118,0.000000,0.026320,0.053072,0.018808,0.037440,0.039132,0.045804,0.044419,0.133333,0.055639,0.025541,0.041761,0.026481,0.035695
6,Cholesterol,0.387126,0.473054,0.389912,0.000000,0.363024,0.238024,0.430447,0.000000,0.300150,0.356287,0.427146,0.377246,0.258982,0.533790,0.311377,0.407186,0.000000
7,Creatinine,0.063306,0.048783,0.072079,0.000000,0.057820,0.060337,0.079545,0.051091,0.086067,0.046976,0.065030,0.041558,0.068246,0.056569,0.057444,0.059394,0.076540
8,DiasABP,0.222288,0.221258,0.223552,0.229701,0.212173,0.235335,0.217357,0.208378,0.226277,0.231468,0.230916,0.218497,0.215240,0.216017,0.220501,0.222197,0.221234
9,FiO2,0.421254,0.407955,0.441474,0.000000,0.431625,0.422815,0.415464,0.454358,0.438086,0.399443,0.399171,0.374807,0.413956,0.412962,0.471615,0.444020,0.479529


<h5>Minimum MAE value in each subgroup</h5>

In [37]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
MechVent
0.0
--------------------
Female
MechVent
0.0
--------------------
Male
MechVent
0.0
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.0
--------------------
-65
MechVent
0.0
--------------------
ICUType 1
MechVent
0.0
--------------------
ICUType 2
MechVent
0.0
--------------------
ICUType 3
MechVent
0.0
--------------------
ICUType 4
MechVent
0.0
--------------------
Undefined classification
MechVent
0.0
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.0
--------------------
Overweight
MechVent
0.0
--------------------
Obesity 1
MechVent
0.0
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
MechVent
0.0
--------------------


<h5> Maximum MAE value in each subgroup </h5>

In [38]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
SaO2
0.9685085946654024
--------------------
Female
SaO2
0.9617415842380362
--------------------
Male
SaO2
0.9675699026353866
--------------------
Undefined Gender
GCS
0.9375000149009268
--------------------
+65
SaO2
0.9675195857723378
--------------------
-65
SaO2
0.9603608350471073
--------------------
ICUType 1
SaO2
0.964683557990219
--------------------
ICUType 2
SaO2
0.9694730503594138
--------------------
ICUType 3
Temp
0.9274873339532732
--------------------
ICUType 4
SaO2
0.9657142966512496
--------------------
Undefined classification
SaO2
0.9598484974015699
--------------------
Low Weight
SaO2
0.966071435383319
--------------------
Normal Weight
SaO2
0.9706250113936478
--------------------
Overweight
SaO2
0.9674803281393501
--------------------
Obesity 1
SaO2
0.9677868954470741
--------------------
Obesity 2
SaO2
0.962777776850541
--------------------
Obesity 3
SaO2
0.9603703794655621
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [39]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax_ori, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 120.21354016661581
ALT : 292.51042278607537
AST : 566.9945693689813
Albumin : 2.9510791413217787
BUN : 25.788438117812007
Bilirubin : 2.7703703989979305
Cholesterol : 157.29999999999214
Creatinine : 1.4927400290198274
DiasABP : 59.684729736472974
FiO2 : 0.5427910102481897
GCS : 11.416553277655188
Glucose : 139.70664855678496
HCO3 : 22.998730468265872
HCT : 30.609207015520653
HR : 86.69561557043218
K : 4.13614858243916
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.9999999999999996
Mg : 2.011890264982128
NIDiasABP : 58.44698860910438
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 138.88944723618073
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 190.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 37.09088744652194
TroponinI : 8.210000050067492
TroponinT : 0.8710493570492538
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 8

In [40]:
df_saits_mae_minmax_ori = toolkits.create_table(testing_mae_saits_variables_minmax_ori, subgroups, variables)

In [41]:
df_saits_mae_minmax_ori 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,120.213540,123.095891,127.046296,0.000000,134.583333,170.847619,89.636364,58.240000,184.601941,121.060607,111.202248,366.500000,117.838710,98.843750,101.571429,63.555556,95.454543
1,ALT,292.510423,118.436620,281.300001,0.000000,218.868130,402.716417,319.000002,497.315789,396.091955,280.655731,291.708863,204.200000,157.095240,641.805538,249.900001,772.285714,1688.333333
2,AST,566.994569,625.684209,250.518520,0.000000,239.448055,430.263396,683.920050,298.000001,491.054948,630.363636,279.442109,889.000068,177.375000,667.714272,388.750004,86.000001,45.250000
3,Albumin,2.951079,2.823188,2.935366,0.000000,2.843836,2.936471,3.340000,2.685714,2.804839,2.770213,2.739744,3.100000,2.636842,3.021053,2.983333,2.750000,2.680000
4,BUN,25.788438,24.924859,29.041096,0.000000,31.607143,22.407408,29.480620,20.337931,33.998246,20.029915,27.157143,30.850000,27.640288,26.492424,30.306667,32.096774,30.428572
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,157.300000,185.999994,158.230769,0.000000,149.250000,107.500000,171.769229,0.000000,128.250000,147.000000,170.666665,154.000000,114.499992,206.285712,132.000000,163.999985,0.000000
7,Creatinine,1.492740,1.173219,1.685744,0.000000,1.372038,1.427404,1.850000,1.224000,1.993464,1.133466,1.530662,1.014286,1.601408,1.344526,1.363768,1.406667,1.783871
8,DiasABP,59.684730,59.403303,60.029592,61.708333,56.923331,63.246397,58.338428,55.887238,60.773501,62.190792,62.040018,58.649750,57.760640,57.972569,59.196723,59.659678,59.396970
9,FiO2,0.542791,0.532284,0.558764,0.000000,0.550984,0.544024,0.538217,0.568943,0.556088,0.525560,0.525345,0.506098,0.537025,0.536240,0.582576,0.560776,0.588828


<h5>Minimum MAE value in each subgroup</h5>

In [42]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [43]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### BRITS

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

In [ ]:
df_brits_mae_standard = toolkits.create_table(testing_mae_brits_variables_standard, subgroups, variables)

In [ ]:
df_brits_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_standard, subgroups, variables) 

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard_ori, subgroups, variables)

In [ ]:
df_brits_mae_standard_ori = toolkits.create_table(testing_mae_brits_variables_standard_ori, subgroups, variables)

In [ ]:
df_brits_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables) 

<h4> MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax, subgroups, variables)

In [ ]:
df_brits_mae_minmax = toolkits.create_table(testing_mae_brits_variables_minmax, subgroups, variables)

In [ ]:
df_brits_mae_minmax 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax, subgroups, variables) 

<h4> MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_brits_mae_minmax_ori = toolkits.create_table(testing_mae_brits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_brits_mae_minmax_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables) 

#### USGAN 

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard, subgroups, variables)

In [ ]:
df_usgan_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [ ]:
df_usgan_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard_ori, subgroups, variables)

In [ ]:
df_usgan_mae_standard_ori = toolkits.create_table(testing_mae_usgan_variables_standard_ori, subgroups, variables)

In [ ]:
df_usgan_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax, subgroups, variables)

In [ ]:
df_usgan_mae_minmax = toolkits.create_table(testing_mae_usgan_variables_minmax, subgroups, variables)

In [ ]:
df_usgan_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

In [ ]:
df_usgan_mae_minmax_ori = toolkits.create_table(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

In [ ]:
df_usgan_mae_minmax_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

#### GPVAE

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard, subgroups, variables)

In [ ]:
df_gpvae_mae_standard = toolkits.create_table(testing_mae_gpvae_variables_standard, subgroups, variables)

In [ ]:
df_gpvae_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_standard_ori = toolkits.create_table(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_standard_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax = toolkits.create_table(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax_ori = toolkits.create_table(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

#### MRNN

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard, subgroups, variables)

In [ ]:
df_mrnn_mae_standard = toolkits.create_table(testing_mae_mrnn_variables_standard, subgroups, variables)

In [ ]:
df_mrnn_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_standard_ori = toolkits.create_table(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax = toolkits.create_table(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax_ori = toolkits.create_table(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)